
<img src="../resources/cropped-SummerWorkshop_Header.png">  

<h1 align="center">EM Connectomics - Question 1</h1> 
<h2 align="center">Summer Workshop on the Dynamic Brain </h2> 
<h3 align="center">Monday, August 29, 2022</h3> 
</div>


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
    <p><b>Task 1:</b> This question explores some of basic functionality needed for investigating the EM dataset.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 1.1:</b> Import the key modules and set parameters

</div>

In [1]:
# Import 
import platform
import os

platstring = platform.platform()
if ('Darwin' in platstring) or ('macOS' in platstring):
    # macOS 
    data_root = "/Volumes/Brain2022/em-connectomics"
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/em-connectomics"
elif ('amzn' in platstring):
    # then on AWS
    data_root = "/data/em-connectomics-data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2022/em-connectomics"
    
# OR if you'd like to override the auto options
# data_root = "./proofread_skeletons/"


In [2]:
# Import modules
from caveclient import CAVEclient

import pandas as pd
import numpy as np
import osx
import matplotlib.pyplot as plt
%matplotlib widget
# Initialize a client for the "minnie65_public_v343" datastack.
datastack_name = 'minnie65_public_v343'
client = CAVEclient(datastack_name)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    <p><b> Task 1.2:</b> From the database, find all the 'cleaned' axons that are predicted to be basket cells. 
      </div>

In [3]:
client.materialize.get_tables()
#client.materialize.get_table_metadata('aibs_soma_nuc_metamodel_preds_v117')

['synapses_pni_2',
 'nucleus_neuron_svm',
 'nucleus_detection_v0',
 'functional_coreg',
 'allen_v1_column_types_slanted',
 'aibs_column_nonneuronal',
 'func_unit_em_match_release',
 'proofreading_status_public_release',
 'aibs_soma_nuc_metamodel_preds_v117']

In [4]:
#get all the clean cells, filter for only those with clean axons
clean_axon_df = client.materialize.query_table('proofreading_status_public_release',
                                             filter_out_dict={'status_axon':['non']})

#get all the soma predictions
soma_predit_df = client.materialize.query_table('aibs_soma_nuc_metamodel_preds_v117')

In [5]:
clean_axon_df.columns

Index(['id', 'valid', 'pt_supervoxel_id', 'pt_root_id', 'valid_id',
       'status_dendrite', 'status_axon', 'pt_position'],
      dtype='object')

In [6]:
soma_predit_df.columns

Index(['id', 'valid', 'target_id', 'classification_system', 'cell_type',
       'id_ref', 'valid__ref', 'volume', 'pt_supervoxel_id', 'pt_root_id',
       'pt_position', 'bb_start_position', 'bb_end_position'],
      dtype='object')

In [7]:
#merge these dataframes to get the clean basket cells

baskets_fromSoma = soma_predit_df[soma_predit_df.cell_type == 'BC']
clean_basket_df = pd.merge(baskets_fromSoma,clean_axon_df,
         on='pt_root_id',
         how='inner', #we want to only care about the cell type for the cells in the synapse dataset  
         suffixes=['_syn', '_soma'])
clean_basket_df

,id_syn,valid_syn,target_id,classification_system,cell_type,id_ref,valid__ref,volume,pt_supervoxel_id_syn,pt_root_id,pt_position_syn,bb_start_position,bb_end_position,id_soma,valid_soma,pt_supervoxel_id_soma,valid_id,status_dendrite,status_axon,pt_position_soma
0,717,t,258292,aibs_neuronal,BC,258292,t,352.823050,88323288895574704,864691135100167712,"[171488, 139312, 21678]","[nan, nan, nan]","[nan, nan, nan]",676,t,88323288895574704,864691135100167712,extended,extended,"[171488, 139312, 21678]"
1,929,t,273805,aibs_neuronal,BC,273805,t,319.236342,88690663285159822,864691135113360025,"[174064, 254832, 22081]","[nan, nan, nan]","[nan, nan, nan]",648,t,88690663285159822,864691135113360025,extended,extended,"[174064, 254832, 22081]"
2,1229,t,267010,aibs_neuronal,BC,267010,t,314.879836,88121390938737418,864691136423071535,"[169584, 207744, 20690]","[nan, nan, nan]","[nan, nan, nan]",643,t,88121390938737418,864691136423071535,extended,extended,"[169584, 207744, 20690]"
3,3279,t,265035,aibs_neuronal,BC,265035,t,248.267571,89173555048399606,864691136011241635,"[177472, 182816, 22269]","[nan, nan, nan]","[nan, nan, nan]",652,t,89173555048399606,864691136011241635,extended,extended,"[177472, 182816, 22269]"
4,3442,t,294749,aibs_neuronal,BC,294749,t,361.953198,91137763852205616,864691137053995126,"[191712, 136928, 22487]","[nan, nan, nan]","[nan, nan, nan]",677,t,91137763852205616,864691137053995126,extended,extended,"[191712, 136928, 22487]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,80961,t,269219,aibs_neuronal,BC,269219,t,335.328707,88404652487728585,864691136100089205,"[172016, 221216, 19783]","[nan, nan, nan]","[nan, nan, nan]",752,t,88404652487728585,864691136100089205,extended,extended,"[172016, 221216, 19783]"
74,83032,t,276440,aibs_neuronal,BC,276440,t,446.436147,89606006917044156,864691135618120847,"[180288, 258960, 23956]","[nan, nan, nan]","[nan, nan, nan]",802,t,89606006917044156,864691135618120847,extended,extended,"[180288, 258960, 23956]"
75,83033,t,276440,aibs_neuronal,BC,276440,t,446.436147,89606006917044156,864691135618120847,"[180288, 258960, 23956]","[nan, nan, nan]","[nan, nan, nan]",802,t,89606006917044156,864691135618120847,extended,extended,"[180288, 258960, 23956]"
76,84351,t,371878,aibs_neuronal,BC,371878,t,281.571983,94455196255712447,864691135341342405,"[215968, 212400, 19590]","[nan, nan, nan]","[nan, nan, nan]",742,t,94455196255712447,864691135341342405,extended,extended,"[215968, 212400, 19590]"


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    <p><b> Task 1.3: </b> Pick an basket cell from the list, and find all its outputs.
        </div>

In [8]:
#client.materialize.query_table('synapses_pni_2').columns

201 - "Limited query to 200000 rows


Index(['id', 'valid', 'pre_pt_supervoxel_id', 'pre_pt_root_id',
       'post_pt_supervoxel_id', 'post_pt_root_id', 'size', 'pre_pt_position',
       'post_pt_position', 'ctr_pt_position'],
      dtype='object')

In [9]:
#choose your favorite basket cell
basket_id = clean_basket_df.pt_root_id[0]

#use the synapse table to get only synapses with your neuron as the pre_id
# all_syn = client.materialize.query_table('synapses_pni_2')

basket_syn = client.materialize.query_table('synapses_pni_2',
                                           filter_equal_dict={'pre_pt_root_id':basket_id},
                                           desired_resolution=(1000,1000,1000))


In [10]:
basket_syn.shape

(4662, 10)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    <p><b> Task 1.4:</b> Visualize this cell in neuroglancer with its inputs and outputs. 
        </div>

In [11]:
#use the nglui helper functions to make a neuroglancer link using the neuron helper
from nglui.statebuilder.helpers import make_neuron_neuroglancer_link
url=make_neuron_neuroglancer_link(client, basket_id, return_as='html')
url


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    <p><b> Task 1.5: </b> Find the 10 largest synapses from this cell
        </div>


In [12]:
basket_syn.columns

Index(['id', 'valid', 'pre_pt_supervoxel_id', 'pre_pt_root_id',
       'post_pt_supervoxel_id', 'post_pt_root_id', 'size', 'pre_pt_position',
       'post_pt_position', 'ctr_pt_position'],
      dtype='object')

In [17]:
#Hint - you can use size to sort your synapse table
sorted_basket_bySize = basket_syn.sort_values('size',ascending = False)
large_10_syns = sorted_basket_bySize[0:9]
large_10_syns

,id,valid,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,post_pt_root_id,size,pre_pt_position,post_pt_position,ctr_pt_position
3350,137072218,t,87689351387069161,864691135100167712,87689351387050765,864691135278547361,77036,"[666.304, 539.192, 762.88]","[666.24, 539.384, 762.5600000000001]","[666.668, 539.352, 762.52]"
809,150464990,t,88670940661060126,864691135100167712,88670940594562228,864691135544713512,44084,"[694.96, 432.81600000000003, 839.76]","[695.112, 432.592, 839.52]","[694.6560000000001, 432.636, 839.44]"
812,130055746,t,87125921404196999,864691135100167712,87125921404200867,864691135307351622,41052,"[650.6800000000001, 524.6800000000001, 1078.96]","[650.408, 524.184, 1079.2]","[650.552, 524.432, 1079.08]"
1977,198665743,t,92333138546040239,864691135100167712,92333138546053735,864691136724959598,30376,"[802.912, 522.624, 714.72]","[802.84, 522.568, 714.72]","[802.7280000000001, 522.1800000000001, 714.72]"
2652,115890612,t,86000157996552577,864691135100167712,85929789252380886,864691136723532541,29908,"[616.64, 528.08, 784.96]","[616.416, 528.344, 784.88]","[616.872, 528.384, 784.9200000000001]"
2313,128153891,t,86844308988485609,864691135100167712,86844308988494293,864691135497612051,29264,"[641.616, 519.312, 1078.04]","[641.808, 519.6, 1078.1200000000001]","[641.616, 519.368, 1077.96]"
2950,97319133,t,84170021093611450,864691135100167712,84170021093630864,864691135644787281,27908,"[563.576, 511.576, 851.48]","[563.408, 511.8, 851.9200000000001]","[563.568, 511.76800000000003, 851.76]"
656,97318184,t,84029352324783377,864691135100167712,84029352324781608,864691136123768102,27260,"[559.944, 514.048, 853.0]","[560.072, 514.376, 853.04]","[560.016, 514.288, 852.8000000000001]"
784,105072496,t,84521864814319928,864691135100167712,84521864814313807,864691135580260804,25684,"[575.264, 512.504, 845.6]","[574.92, 512.104, 845.5600000000001]","[575.056, 512.36, 845.32]"


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    <p><b> Task 1.6:</b> Visualize the cell and these 10 synapses. Examine the post-synaptic target at each of these 10 and note what you notice you about the cell type and location of those synapses.
        </div>

In [25]:
from nglui.statebuilder.helpers import make_synapse_neuroglancer_link
# how about those 6 synapses we found earlier

url = make_synapse_neuroglancer_link(large_10_syns, client, return_as='html')
url

In [20]:
soma_pos_df

,id,valid,target_id,classification_system,cell_type,id_ref,valid__ref,volume,pt_supervoxel_id,pt_root_id,pt_position,bb_start_position,bb_end_position
754,717,t,258292,aibs_neuronal,BC,258292,t,352.82305,88323288895574704,864691135100167712,"[171488, 139312, 21678]","[nan, nan, nan]","[nan, nan, nan]"
